# Thief Detector
## This task tests your Image Processing skills to build a motion detection algorithm that alarms you when you have an unwanted visitor in your home.

## Steps
- 1. Get the live video feed from your webcam
- 2. Fix a scene (the place you want to monitor) and store it as a reference background image
    - Store the first frame as the reference background frame
- 3. For every frame, check if there is any unwanted object inside the scene you are monitoring
    - Use **Background Subtraction** concept (**cv2.absdiff( )**)
        - Subtract the current frame from the reference background image(frame) to see the changes in the scene
        - If there is enormous amount of pixels distrubed in the subtraction result image
            - unwanted visitor (place is unsafe --> alarm the authorities)
        - If there is no enormous amount of pixels distrubed in the subtraction result image
            - no unwanted visitor (place is safe)
- 4. Output the text **"UNSAFE"** in **red** color on the top right of the frame when there is an intruder in the scene.
- 5. Save the live feed
- 6. Submit the (.ipynb) file

In [ ]:
import cv2
import numpy as np
import os

print("Current working directory:", os.getcwd())

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('Abdul_rasaq_session_output.mp4',
                      fourcc, 
                      20.0,
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                       int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

num_frames_to_average = 30
frame_count = 0
background_frames = []

print("Calibrating background. Please keep the scene still...")

#
while frame_count < num_frames_to_average:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame during calibration.")
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        exit()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (21, 21), 0)
    background_frames.append(gray_blur)
    frame_count += 1
    
    percent_done = int((frame_count / num_frames_to_average) * 100)
    cv2.putText(frame, f"Calibrating: {percent_done}%", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.imshow("Live Feed", frame)
    cv2.waitKey(1)


background_gray = np.mean(background_frames, axis=0).astype(np.uint8)

print("Motion Detector Active. Press 'q' to quit.")

MIN_AREA = 1000        
THRESHOLD_VALUE = 30   
FRAMES_TO_CONFIRM = 3  

consecutive_motion_frames = 0
is_unsafe = False

frame_counter = 0
update_background_every = 100  
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (21, 21), 0)
   
    frame_delta = cv2.absdiff(background_gray, gray_blur)
    
    
    thresh = cv2.threshold(frame_delta, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    
  
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    current_motion = False
    
   
    for contour in contours:
        if cv2.contourArea(contour) >= MIN_AREA:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            current_motion = True
    
 
    if current_motion:
        consecutive_motion_frames += 1
    else:
        consecutive_motion_frames = 0
    
   
    if consecutive_motion_frames >= FRAMES_TO_CONFIRM:
        is_unsafe = True
    elif consecutive_motion_frames == 0:
        is_unsafe = False
    
    if is_unsafe:
        cv2.putText(frame, "UNSAFE", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame, "SAFE", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    
    cv2.imshow("Live Feed", frame)
    
   
    out.write(frame)
    
   
    frame_counter += 1
    if frame_counter >= update_background_every:
        alpha = 0.05  
       
        background_gray = cv2.addWeighted(gray_blur, alpha, background_gray, 1 - alpha, 0)
        frame_counter = 0
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Session ended. Video saved as Abdul_rasaq_session_output.mp4 in:", os.getcwd())


Current working directory: /Users/omisesan/Desktop/Desktop_Omisesan/APPLIED_AI _DEV/MACHINE LEARNING II/Exam
Calibrating background. Please keep the scene still...


2025-03-28 21:24:22.614 python[29180:361487] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-28 21:24:22.614 python[29180:361487] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Motion Detector Active. Press 'q' to quit.
Session ended. Video saved as Abdul_rasaq_session_output.mp4 in: /Users/omisesan/Desktop/Desktop_Omisesan/APPLIED_AI _DEV/MACHINE LEARNING II/Exam


: 

## Read first frame, convert to Grayscale and store it as reference background image [10 points]

## Compute Absolute Difference between Current and First frame [20 points]

## Apply threshold [5 points]

## Find contours [10 points]

## Check if contourArea is large and draw rectangle around the object, output "UNSAFE" text in red color [30 points]

## Display images [10 points]

## Release objects [5 points]